# ITR Tool - Quick Temperature Score Calculation
This notebook provides a simple example of the ITR Toolkit. It shows how to use it to calculate the temperature score for companies, aggregate them to a portfolio level to get the portfolio temperature score. 


### Getting started
Make sure you are running the notebook with the requirements installed available in the example folder

In [1]:
!pip install -r requirements.txt

Looking in indexes: http://rdnexus1.ortec.finance:8081/nexus/repository/pypi-all/simple


In [2]:
import ITR
from ITR.data.excel import ExcelProvider
from ITR.portfolio_aggregation import PortfolioAggregationMethod
# from ITR.portfolio_coverage_tvp import PortfolioCoverageTVP
from ITR.temperature_score import TemperatureScore
# from ITR.target_validation import TargetProtocol
from ITR.interfaces import ETimeFrames, EScope
import pandas as pd

## Download the dummy data provider
We have prepared dummy data for you to be able to run the tool as it is to familiarise yourself with how it works. To use your own data; please check out to the [Data Requirements section](https://os-c.github.io/ITR/DataRequirements.html) of the technical documentation for more details on data requirements and formatting. 

*The dummy data may include some company names, but the data associated with those company names is completely random and any similarities with real world data is purely coincidental. 


In [3]:
import urllib.request
import os

if not os.path.isdir("data"):
    os.mkdir("data")
if not os.path.isfile("data/data_provider_example.xlsx"):
    urllib.request.urlretrieve("https://github.com/os-c/ITR/tree/project-template/examples/data/data_provider_example.xlsx", "data/data_provider_example.xlsx")
if not os.path.isfile("data/example_portfolio.csv"):
    urllib.request.urlretrieve("https://github.com/os-c/ITR/tree/project-template/examples/data/example_portfolio.csv", "data/example_portfolio.csv")

##### Logging
The ITR module uses the Python standard library logging utilities to send log messages. The log level can be changed according to the user's needs.

In [4]:
import logging
root_logger = logging.getLogger()
root_logger.setLevel("INFO")

## Create a data provider
Data providers let you connect to the data source of your choice. In this case we are connecting to Excel as a data provider. 

In [5]:
provider = ExcelProvider(path="data/data_provider_example.xlsx")

## Load your portfolio
In our case the portfolio is stored as a CSV file. The portfolio should at least have an "id" (the identifier of the company) and a "proportion" (the weight of the company in your portfolio e.g. the value of the shares you hold) column.

Please see the technical documentation on [Data Legends](https://ofbdabv.github.io/SBTi/Legends.html#) for details on data requirements. 

In [6]:
df_portfolio = pd.read_csv("data/example_portfolio.csv", encoding="iso-8859-1", sep=';')

In [7]:
df_portfolio.head(5)

,company_name,company_id,company_isin,investment_value
0,Company AG,US0079031078,US0079031078,35000000
1,Company AH,US00724F1012,US00724F1012,10000000
2,Company AI,FR0000125338,FR0000125338,10000000
3,Company AJ,US17275R1023,US17275R1023,10000000
4,Company AK,CH0198251305,CH0198251305,10000000


To load the data from the data provider, we have to pass a list of IPortfolioCompany instances. The module has a strict [data model](https://ofbdabv.github.io/SBTi/autoapi/SBTi/interfaces/index.html) to convert Pandas Dataframe to the right object types we supplied a utility function.


In [8]:
companies = ITR.utils.dataframe_to_portfolio(df_portfolio)

## Calculate the temperature scores
In the amended portfolio you'll find your original portfolio, amended with the emissions and the temperature score.

In [9]:
temperature_score = TemperatureScore(                  # all available options:
    time_frames=list(ITR.interfaces.ETimeFrames),     # ETimeFrames:  LONG
    scopes=[EScope.S1S2],    
    aggregation_method=PortfolioAggregationMethod.WATS # Options for the aggregation method are WATS, TETS, AOTS, MOTS, EOTS, ECOTS, and ROTS.
)
amended_portfolio = temperature_score.calculate(data_providers=[provider], portfolio=companies)

For every company the tool assigns a score for all the requested timeframe and scope combinations. In this example we used the full set resulting in 9 scores per company as displayed below:

In [10]:
amended_portfolio[['company_name', 'time_frame', 'scope', 'temperature_score']].head(9)

NameError: name 'amended_portfolio' is not defined

## Calculate the aggregated temperature score
Calculate an aggregated temperature score. This can be done using different aggregation methods. Here we'll use the "Weighted Average Temperature Score" (WATS) by initializing the TemperatureScore Object with PortfolioAggregationMethod.WATS. The temperature scores are calculated per time-frame/scope combination.


In [ ]:
aggregated_scores = temperature_score.aggregate_scores(amended_portfolio)

In [ ]:
# Here we cast a ScoreAggregation object to a Pandas Dataframe for viewing the temp scores in a human readable way
pd.DataFrame(aggregated_scores.dict()).applymap(lambda x: round(x['all']['score'], 2))